In [19]:
import pandas as pd
from datetime import date, timedelta
import urllib
import numpy as np
import altair as alt
from functools import reduce
import untangle
import json

In [3]:
xml = untangle.parse('https://publicdashacc.blob.core.windows.net/publicdata?restype=container&comp=list')

In [4]:
xml

Element(name = None, attributes = None, cdata = )

In [5]:
blobs = xml.EnumerationResults.Blobs.Blob

In [6]:
for blob in blobs:
    print(blob.Name.cdata)

countries.geojson
data_20200409.json
data_202004091537.json
data_202004101527.json
data_202004111452.json
data_202004121411.json
data_202004131413.json
data_202004141435.json
data_202004141544.json
data_202004151454.json
data_202004161444.json
regions.geojson
utlas.geojson


In [7]:
jsons = sorted(
    list(
        filter(
            lambda b: b.endswith('.json'),
            map(
                lambda b: b.Name.cdata,
                blobs
            )
        )
    )
)

latest_json = jsons[-1]
print(latest_json)

data_202004161444.json


In [8]:
host = 'https://c19pub.azureedge.net/'
latest_json_url = host + latest_json
print(latest_json_url)

https://c19pub.azureedge.net/data_202004161444.json


In [16]:
df = pd.read_json(latest_json_url)
df

,lastUpdatedAt,disclaimer,overview,countries,regions,utlas
K02000001,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,"{'name': {'value': 'United Kingdom'}, 'totalCa...",NaN,NaN,NaN
E92000001,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,"{'name': {'value': 'England'}, 'totalCases': {...",NaN,NaN
S92000003,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,"{'name': {'value': 'Scotland'}, 'totalCases': ...",NaN,NaN
N92000002,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,"{'name': {'value': 'Northern Ireland'}, 'total...",NaN,NaN
W92000004,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,"{'name': {'value': 'Wales'}, 'totalCases': {'v...",NaN,NaN
...,...,...,...,...,...,...
E08000015,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,NaN,NaN,"{'name': {'value': 'Wirral'}, 'totalCases': {'..."
E06000041,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,NaN,NaN,"{'name': {'value': 'Wokingham'}, 'totalCases':..."
E08000031,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,NaN,NaN,"{'name': {'value': 'Wolverhampton'}, 'totalCas..."
E10000034,2020-04-16T14:44:18.371573Z,Case counts for England and subnational areas ...,NaN,NaN,NaN,"{'name': {'value': 'Worcestershire'}, 'totalCa..."


In [10]:
utlas = df[df['utlas'].notna()]['utlas']
utlas.reset_index()

,index,utlas
0,E09000002,"{'name': {'value': 'Barking and Dagenham'}, 't..."
1,E09000003,"{'name': {'value': 'Barnet'}, 'totalCases': {'..."
2,E08000016,"{'name': {'value': 'Barnsley'}, 'totalCases': ..."
3,E06000022,{'name': {'value': 'Bath and North East Somers...
4,E06000055,"{'name': {'value': 'Bedford'}, 'totalCases': {..."
...,...,...
145,E08000015,"{'name': {'value': 'Wirral'}, 'totalCases': {'..."
146,E06000041,"{'name': {'value': 'Wokingham'}, 'totalCases':..."
147,E08000031,"{'name': {'value': 'Wolverhampton'}, 'totalCas..."
148,E10000034,"{'name': {'value': 'Worcestershire'}, 'totalCa..."


In [11]:
wolverhampton_json = utlas['E08000031']
wolverhampton_json

{'name': {'value': 'Wolverhampton'},
 'totalCases': {'value': 501},
 'dailyConfirmedCases': [{'date': '2020-03-05', 'value': 1},
  {'date': '2020-03-07', 'value': 3},
  {'date': '2020-03-08', 'value': 1},
  {'date': '2020-03-10', 'value': 3},
  {'date': '2020-03-11', 'value': 5},
  {'date': '2020-03-12', 'value': 5},
  {'date': '2020-03-13', 'value': 11},
  {'date': '2020-03-14', 'value': 8},
  {'date': '2020-03-15', 'value': 3},
  {'date': '2020-03-16', 'value': 4},
  {'date': '2020-03-17', 'value': 15},
  {'date': '2020-03-18', 'value': 6},
  {'date': '2020-03-19', 'value': 7},
  {'date': '2020-03-20', 'value': 15},
  {'date': '2020-03-21', 'value': 11},
  {'date': '2020-03-22', 'value': 5},
  {'date': '2020-03-23', 'value': 9},
  {'date': '2020-03-24', 'value': 24},
  {'date': '2020-03-25', 'value': 11},
  {'date': '2020-03-26', 'value': 13},
  {'date': '2020-03-27', 'value': 26},
  {'date': '2020-03-28', 'value': 18},
  {'date': '2020-03-29', 'value': 12},
  {'date': '2020-03-30', 

In [12]:
wolverhampton = pd.json_normalize(wolverhampton_json)
wolverhampton

,dailyConfirmedCases,dailyTotalConfirmedCases,name.value,totalCases.value
0,"[{'date': '2020-03-05', 'value': 1}, {'date': ...","[{'date': '2020-03-05', 'value': 1}, {'date': ...",Wolverhampton,501


In [13]:
wolverhampton['dailyTotalConfirmedCases']

0    [{'date': '2020-03-05', 'value': 1}, {'date': ...
Name: dailyTotalConfirmedCases, dtype: object

In [14]:
list(wolverhampton['dailyTotalConfirmedCases'])

[[{'date': '2020-03-05', 'value': 1},
  {'date': '2020-03-07', 'value': 4},
  {'date': '2020-03-08', 'value': 5},
  {'date': '2020-03-10', 'value': 8},
  {'date': '2020-03-11', 'value': 13},
  {'date': '2020-03-12', 'value': 18},
  {'date': '2020-03-13', 'value': 29},
  {'date': '2020-03-14', 'value': 37},
  {'date': '2020-03-15', 'value': 40},
  {'date': '2020-03-16', 'value': 44},
  {'date': '2020-03-17', 'value': 59},
  {'date': '2020-03-18', 'value': 65},
  {'date': '2020-03-19', 'value': 72},
  {'date': '2020-03-20', 'value': 87},
  {'date': '2020-03-21', 'value': 98},
  {'date': '2020-03-22', 'value': 103},
  {'date': '2020-03-23', 'value': 112},
  {'date': '2020-03-24', 'value': 136},
  {'date': '2020-03-25', 'value': 147},
  {'date': '2020-03-26', 'value': 160},
  {'date': '2020-03-27', 'value': 186},
  {'date': '2020-03-28', 'value': 204},
  {'date': '2020-03-29', 'value': 216},
  {'date': '2020-03-30', 'value': 237},
  {'date': '2020-03-31', 'value': 260},
  {'date': '2020-04

In [17]:
for daylist in wolverhampton['dailyTotalConfirmedCases']:
    for day in daylist:
        print(day)

{'date': '2020-03-05', 'value': 1}
{'date': '2020-03-07', 'value': 4}
{'date': '2020-03-08', 'value': 5}
{'date': '2020-03-10', 'value': 8}
{'date': '2020-03-11', 'value': 13}
{'date': '2020-03-12', 'value': 18}
{'date': '2020-03-13', 'value': 29}
{'date': '2020-03-14', 'value': 37}
{'date': '2020-03-15', 'value': 40}
{'date': '2020-03-16', 'value': 44}
{'date': '2020-03-17', 'value': 59}
{'date': '2020-03-18', 'value': 65}
{'date': '2020-03-19', 'value': 72}
{'date': '2020-03-20', 'value': 87}
{'date': '2020-03-21', 'value': 98}
{'date': '2020-03-22', 'value': 103}
{'date': '2020-03-23', 'value': 112}
{'date': '2020-03-24', 'value': 136}
{'date': '2020-03-25', 'value': 147}
{'date': '2020-03-26', 'value': 160}
{'date': '2020-03-27', 'value': 186}
{'date': '2020-03-28', 'value': 204}
{'date': '2020-03-29', 'value': 216}
{'date': '2020-03-30', 'value': 237}
{'date': '2020-03-31', 'value': 260}
{'date': '2020-04-01', 'value': 279}
{'date': '2020-04-02', 'value': 296}
{'date': '2020-04-03

In [20]:
json.loads

<function json.loads(s, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>